<a href="https://colab.research.google.com/github/Igor-Belskiy/netology-case/blob/main/%D0%9D%D0%B5%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D1%8F_(%D0%BC%D0%BE%D1%91_%D1%80%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D0%B5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка библиотек и файлов данных

In [ ]:
import numpy as np   # для работы с данными
import pandas as pd   # для работы с таблицами
import time    # Импортируем библиотеку time
from datetime import datetime    # Для обработки дат
import re

import matplotlib.pyplot as plt #Отрисовка графиков

from tensorflow.keras.preprocessing.text import Tokenizer   # Методы для работы с текстами и преоборазвания их в последовательности
from tensorflow.keras import utils  # для работы с категориальными данными
from tensorflow.keras.optimizers import Adam #Оптимизатор
from tensorflow.keras.models import Sequential, Model #Два варианты моделей
from tensorflow.keras.layers import concatenate, Input, Dense, Dropout, BatchNormalization, Flatten, Conv1D, Conv2D, LSTM, GlobalMaxPooling1D, MaxPooling1D, RepeatVector #Стандартные слои
from tensorflow.keras.utils import plot_model     # Для отрисовки моделей

from sklearn.preprocessing import StandardScaler, MinMaxScaler #Нормировщики
from sklearn.model_selection import train_test_split  # Для раздения выборки на тестовую и обучающую

In [ ]:
!rm -R '/content/data' # Проверяем наличие папки data. Если есть, удаляем ее

rm: cannot remove '/content/data': No such file or directory


In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd   # для загрузки с гугл-драйва в колабораторию

# Копию бучающег и тестовог датасета предварительно загрузил к себе на гугл-диск
# и Колабораторию загружаю их напрямую уже со своего диска (досутуп отркыт для всех по ссылке)

gdd.download_file_from_google_drive(file_id='1WhW6MrKYgs_5K3gE-iPIRLJ3EZr09hnm', 
                                    dest_path='/content/data/test_dataset_test.csv', 
                                    showsize=True,
                                    overwrite=True) #Скачиваем тестовый датасет

gdd.download_file_from_google_drive(file_id='1XQ6wqeeOvi0xgAELvfhIN902kX16bsTa', 
                                    dest_path='/content/data/train_dataset_train.csv', 
                                    showsize=True,
                                    overwrite=True) #Скачиваем тренировочный датасет

13.3 MiB Done.
31.8 MiB Done.


In [ ]:
gdd.download_file_from_google_drive(file_id='1aw5ykTUO15Z7FtXzGIxOtCOJghSP7uD6', 
                                    dest_path='/content/data/sample_solution.csv', 
                                    showsize=True,
                                    overwrite=True) #Скачиваем пример решения

704.0 KiB Done.


In [ ]:
df_sample_solution = pd.read_csv('/content/data/sample_solution.csv', index_col= 0)

In [ ]:
df_test = pd.read_csv('/content/data/test_dataset_test.csv', index_col= 0)

df_train = pd.read_csv('/content/data/train_dataset_train.csv', index_col= 0)

 # Осмотр данных

In [ ]:
df_sample_solution.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84997 entries, 77551 to 95178
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   target  84997 non-null  int64
dtypes: int64(1)
memory usage: 1.3 MB


In [ ]:
df_sample_solution.sample(5)

,target
id,
112112,0
46938,0
108355,0
230544,0
192790,0


In [ ]:
df_train['target'].value_counts()

0    174301
1     13512
5      3678
4      3659
3      3027
2      1823
Name: target, dtype: int64

In [ ]:
for x in df_test.columns:

    print('\nСтолбец:', x) 
    print('Тип:', df_test[x].dtypes, '\n')
    print(df_test[x].value_counts())
    print('\nКоличество уникальных значений:', df_test[x].nunique())
    print('\nКоличество пропущенных значений (True - пропущенные, False - заполненные):\n',df_test[x].isna().value_counts())
    print('------------------------------------------------------------------------------------------------------------------------------------------------\n')


Столбец: age_indicator
Тип: float64 

19.0    1766
23.0    1751
3.0     1748
33.0    1744
34.0    1739
21.0    1739
37.0    1738
17.0    1731
7.0     1722
2.0     1717
28.0    1711
10.0    1711
11.0    1709
29.0    1708
31.0    1707
27.0    1705
13.0    1703
30.0    1703
35.0    1701
26.0    1701
4.0     1695
16.0    1695
14.0    1692
36.0    1692
9.0     1690
39.0    1689
1.0     1681
22.0    1679
15.0    1676
38.0    1675
20.0    1669
8.0     1666
24.0    1664
5.0     1661
6.0     1658
32.0    1646
12.0    1643
25.0    1640
18.0    1599
40.0    1384
Name: age_indicator, dtype: int64

Количество уникальных значений: 40

Количество пропущенных значений (True - пропущенные, False - заполненные):
 False    67548
True     17449
Name: age_indicator, dtype: int64
------------------------------------------------------------------------------------------------------------------------------------------------


Столбец: month_id
Тип: object 

11/30/2021    10101
10/31/2021     9360
9/30/2021  

In [ ]:
for x in df_train.columns:

    print('\nСтолбец:', x) 
    print('Тип:', df_train[x].dtypes, '\n')
    print(df_train[x].value_counts())
    print('\nКоличество уникальных значений:', df_train[x].nunique())
    print('\nКоличество пропущенных значений (True - пропущенные, False - заполненные):\n',df_train[x].isna().value_counts())
    print('------------------------------------------------------------------------------------------------------------------------------------------------\n')


Столбец: age_indicator
Тип: float64 

25.0    4119
15.0    4066
8.0     4058
32.0    4057
12.0    4057
38.0    4047
39.0    4043
6.0     4043
35.0    4041
1.0     4040
30.0    4034
9.0     4034
22.0    4032
36.0    4031
4.0     4030
33.0    4023
13.0    4023
16.0    4020
10.0    4016
2.0     4013
28.0    4013
24.0    4012
31.0    4004
11.0    4002
29.0    4001
21.0    4000
27.0    3996
20.0    3983
7.0     3977
14.0    3972
26.0    3968
5.0     3959
17.0    3955
37.0    3951
23.0    3948
3.0     3940
19.0    3932
34.0    3887
18.0    3633
40.0    3163
Name: age_indicator, dtype: int64

Количество уникальных значений: 40

Количество пропущенных значений (True - пропущенные, False - заполненные):
 False    159123
True      40877
Name: age_indicator, dtype: int64
------------------------------------------------------------------------------------------------------------------------------------------------


Столбец: month_id
Тип: object 

11/30/2021    23987
10/31/2021    21659
9/30/2021

In [ ]:
df_train.sample(5).transpose()

id,17838,149902,115559,25580,199451
age_indicator,15.0,9.0,26.0,8.0,NaN
month_id,10/31/2020,9/30/2021,7/31/2021,1/31/2021,11/30/2021
student_id,6563120,6744206,5992898,6673303,5936412
program_id,1543,1209,242,887,1287
carts_created_at,9/14/2020,8/23/2020,9/1/2020,7/3/2020,4/15/2021
spent_time_total,NaN,NaN,NaN,NaN,NaN
spent_time_to_complete_hw,NaN,NaN,NaN,NaN,NaN
completed_hw,NaN,NaN,NaN,NaN,NaN
failed_hw,NaN,NaN,NaN,NaN,NaN
reworked_hw,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 15182 to 162913
Data columns (total 58 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   age_indicator              159123 non-null  float64
 1   month_id                   200000 non-null  object 
 2   student_id                 200000 non-null  int64  
 3   program_id                 200000 non-null  int64  
 4   carts_created_at           200000 non-null  object 
 5   spent_time_total           86309 non-null   float64
 6   spent_time_to_complete_hw  42467 non-null   float64
 7   completed_hw               97599 non-null   float64
 8   failed_hw                  97599 non-null   float64
 9   reworked_hw                97599 non-null   float64
 10  interacted_hw              97599 non-null   float64
 11  avg_hw_mark                44496 non-null   float64
 12  test_with_good_mark        97599 non-null   float64
 13  test_with_great_mark     

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84997 entries, 77551 to 95178
Data columns (total 57 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   age_indicator              67548 non-null  float64
 1   month_id                   84997 non-null  object 
 2   student_id                 84997 non-null  int64  
 3   program_id                 84997 non-null  int64  
 4   carts_created_at           84997 non-null  object 
 5   spent_time_total           36687 non-null  float64
 6   spent_time_to_complete_hw  18046 non-null  float64
 7   completed_hw               41481 non-null  float64
 8   failed_hw                  41481 non-null  float64
 9   reworked_hw                41481 non-null  float64
 10  interacted_hw              41481 non-null  float64
 11  avg_hw_mark                18909 non-null  float64
 12  test_with_good_mark        41481 non-null  float64
 13  test_with_great_mark       41481 non-null 

# Кодируем выход методом One Hot Encoding

In [ ]:
df_train['target'].value_counts()

0    174301
1     13512
5      3678
4      3659
3      3027
2      1823
Name: target, dtype: int64

In [ ]:
y_train = utils.to_categorical(df_train['target'], 6)

In [ ]:
y_train[10050]

array([0., 0., 0., 0., 1., 0.], dtype=float32)

In [ ]:
df_train['target'].iloc[10050]

4

#Экспериментируем с заполнением пропущенных данных

In [ ]:
df_train.columns

Index(['age_indicator', 'month_id', 'student_id', 'program_id',
       'carts_created_at', 'spent_time_total', 'spent_time_to_complete_hw',
       'completed_hw', 'failed_hw', 'reworked_hw', 'interacted_hw',
       'avg_hw_mark', 'test_with_good_mark', 'test_with_great_mark',
       'webinars', 'avg_quiz_result', 'notes', 'hw_leader', 'lessons',
       'activity', 'bought_d1', 'bought_d2', 'bought_d3', 'bought_d4',
       'bought_d5', 'bought_avg_duration', 'payment_type', 'promo', 'price',
       'communication_type', 'auto_payment', 'ABC', 'city', 'country',
       'gender', 'speed_recall', 'os', 'browser', 'platform',
       'm_avg_talk_duration', 'm_avg_duration', 'm_missed_calls',
       'm_total_calls', 'm_was_conversations', 'm_total_duration',
       'p_avg_talk_duration', 'p_avg_duration', 'p_missed_calls',
       'p_total_calls', 'p_was_conversations', 'p_total_duration',
       'support_feedback_avg', 'feedback_avg_d1', 'feedback_avg_d2',
       'feedback_avg_d3', 'feedback_

In [ ]:
df_train['student_id'].isna().value_counts()

False    200000
Name: student_id, dtype: int64

In [ ]:
df_train['student_id'].nunique()

31153

# Готовим данные в таблицу для анализа

In [ ]:
s = df_train.sample(3).transpose()

s.insert(0,'type', df_train.dtypes)  

s.insert(1, 'notnull', df_train.notnull().sum())

s.to_excel('types.xls')

s

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  import sys


id,type,notnull,62002,131294,38142
age_indicator,float64,159123,NaN,NaN,23.0
month_id,object,200000,4/30/2021,9/30/2021,1/31/2021
student_id,int64,200000,7021669,5557843,6992918
program_id,int64,200000,1605,1153,1160
carts_created_at,object,200000,2/2/2021,7/20/2020,1/4/2021
spent_time_total,float64,86309,8.0,NaN,22.0
spent_time_to_complete_hw,float64,42467,NaN,NaN,NaN
completed_hw,float64,97599,0.0,NaN,0.0
failed_hw,float64,97599,0.0,NaN,0.0
reworked_hw,float64,97599,0.0,NaN,0.0


# Готовим файл случайный результат для проверки выгрузки на платформу

In [ ]:
# Готовим случайный вектор для тестиования выхода сети
rng = np.random.default_rng()
rints = rng.integers(low=0, high=6, size=df_test.shape[0])

rints

array([1, 4, 5, ..., 2, 4, 1])

In [ ]:
df_result = df_test.copy(deep=True)

In [ ]:
# Пробуем заполнять файл результата случайными данными.

df_result.insert(0, 'target', rints)

df_result = df_result.drop(df_test.columns, axis = 1)

df_result.to_csv('result_1.csv')